In [1]:
data_var = '2023-12-14'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8223,2023-12-14,Brasil Nbb,19:00,Corinthians Paulista,Minas,2.93,1.37,158.5,1.85,1.85,5.5,1.94,1.77,YJDLRnXj,0.341297,0.729927,0.540541,0.540541,0.071224,0.8,0.4,NaN,NaN,138.190,40.596180,0.293771,112.248,26.689802,0.237775,119.838,111.510,104.06,133.30,0.0,0.0,0.0,0.0,0.513064,0.00000,0.064802,-1.79,-0.358,-5.391061,0.000000,0.0,0.000000,1.64,0.328,1.128049,0.000000,0.0,0.000000
8224,2023-12-14,Eua Nba,21:30,Boston Celtics,Cleveland Cavaliers,1.30,4.10,224.5,1.80,2.09,-9.5,2.06,1.86,EPwCSGbs,0.769231,0.243902,0.555556,0.478469,0.013133,0.8,0.4,NaN,NaN,156.078,20.588594,0.131912,271.892,185.543721,0.682417,156.078,316.612,145.20,600.03,0.0,1.0,0.0,0.0,0.733296,0.10543,0.072154,1.33,0.266,1.127820,0.787992,0.9,0.112008,1.67,0.334,9.281437,0.514681,0.5,-0.014681
8225,2023-12-14,Europa Euroliga,15:00,Crvena zvezda,Alba Berlin,1.19,4.91,164.5,1.91,1.91,-11.5,2.10,1.67,6gXUm53o,0.840336,0.203666,0.523560,0.523560,0.044002,0.0,0.0,NaN,NaN,181.208,47.914820,0.264419,448.618,330.198609,0.736035,0.000,0.000,163.93,518.40,0.0,0.0,0.0,0.0,0.862438,0.00000,0.161303,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8226,2023-12-14,Europa Euroliga,15:00,Zalgiris Kaunas,Partizan,1.87,1.95,159.5,1.91,1.91,-1.5,1.95,1.87,YaJ7W3YA,0.534759,0.512821,0.523560,0.523560,0.047580,0.0,0.0,NaN,NaN,202.200,97.572077,0.482552,151.628,43.348704,0.285889,0.000,0.000,198.29,219.02,0.0,0.0,0.0,0.0,0.029617,0.00000,0.029617,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8227,2023-12-14,Europa Euroliga,16:05,Maccabi Tel Aviv,Anadolu Efes,1.41,3.03,171.5,1.91,1.91,-7.5,2.05,1.69,KdF3XqJ4,0.709220,0.330033,0.523560,0.523560,0.039253,0.6,1.0,NaN,NaN,165.152,52.620248,0.318617,181.326,17.208887,0.094906,148.776,199.906,115.26,161.99,0.0,0.0,0.0,0.0,0.515997,0.00000,0.136128,-2.20,-0.440,-0.931818,0.000000,0.0,0.000000,-0.16,-0.032,-63.437500,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8297,2023-12-14,Rússia Superliga,12:00,Zenit Petersburg 2,Barnaul,2.02,1.72,154.5,1.85,1.85,1.5,1.87,1.83,4GPfML2G,0.495050,0.581395,0.540541,0.540541,0.076445,0.0,0.0,NaN,NaN,NaN,NaN,NaN,129.520,16.872411,0.130269,0.000,0.000,124.20,138.61,0.0,0.0,0.0,0.0,0.113440,0.00000,0.015289,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8298,2023-12-14,Rússia Superliga,13:00,CSKA Moscow 2,Novosibirsk,2.53,1.48,153.5,1.85,1.85,3.5,1.97,1.73,r7ObLuIM,0.395257,0.675676,0.540541,0.540541,0.070933,0.6,0.0,NaN,NaN,151.238,41.791326,0.276328,142.262,62.526209,0.439514,136.306,0.000,184.44,108.80,0.0,0.0,0.0,0.0,0.370305,0.00000,0.091733,0.02,0.004,382.500000,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8299,2023-12-14,Rússia Premier League Feminina,10:00,Neftyanik Omsk F,UMMC Ekaterinburg F,11.60,1.02,133.5,1.85,1.85,16.5,2.09,1.63,O0ImTmLg,0.086207,0.980392,0.540541,0.540541,0.066599,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,132.72,121.80,0.0,0.0,0.0,0.0,1.185609,0.00000,0.174876,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8300,2023-12-14,Rússia Premier League Feminina,11:00,Nadezhda F,Samara F,1.28,3.44,135.5,1.85,1.85,-9.5,2.02,1.69,8SGGzWSJ,0.781250,0.290698,0.540541,0.540541,0.071948,0.0,0.0,NaN,NaN,NaN,Na

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
1,21:30,Eua Nba,Boston Celtics,Cleveland Cavaliers,224.5,1.80,1.0000,Over
3,15:00,Europa Euroliga,Zalgiris Kaunas,Partizan,159.5,1.91,0.7448,Over
57,21:30,Eua Nba,Miami Heat,Chicago Bulls,215.5,1.80,1.0000,Over
61,20:30,Eua Ncaa,Youngstown State,Bethany,151.5,1.91,0.5453,Over
